In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import re
import pandas as pd
import openai
import os
import json
# from gpt3_sandbox.api.gpt import GPT
# from gpt3_sandbox.api.gpt import Example
from pandasql import sqldf
from tqdm import tqdm
import numpy as np
from GptPrompter import *
from GptCOTPrompter import *
from AutoReasoner import *
import dotenv
import traceback

config = dotenv.dotenv_values(".env")
openai.api_key = config['OPENAI_API_KEY']

dataset = pd.read_csv('./dataset/WikiTableQuestions/data/pristine-unseen-tables.tsv', sep='\t')
# dataset = pd.read_csv('./dataset/WikiTableQuestions/data/training.tsv', sep='\t')

In [3]:
import fasttext
ft = fasttext.load_model('cc.en.300.bin')

In [53]:
# TODO: 1) add more examples for python, 2) balance the length, 3) split the steps, first step SQL etc

NNDemo = False
max_demo = 6
template = 'original-sql-py'
# template = 'formatv1-sql-py'
def parallel_codex_func_formatv1(i):
    max_retry = 3
    while max_retry>0:
        try:
            codex_prompter = CodexAnswerCOTExecutor_decisionAction(
                                              f'prompt_template/{template}-DA.json',
                                              dataset.iloc[i]['id'], 
                                              dataset.iloc[i]['utterance'], 
                                              dataset.iloc[i]['context'], 
                                              dataset.iloc[i]['targetValue'],  
                                              base_path='./dataset/WikiTableQuestions/',
                                              demo_file=f'few-shot-demo/WikiTQ-{program}.json',
                                             )
            codex_prompter.max_demo = max_demo
            codex_prompter._gen_gpt_prompt()
            codex_prompter._get_gpt_prediction()
            log = codex_prompter._log_dict()
            break
        except Exception as e:
            log = {
                'id': dataset.iloc[i]['id'],
                'uncaught_err': str(e)
            }
            max_retry -= 1
    return log
    
for program in [ 'sql-py']:
    n_threads = 10
    maxLimit = float('inf')
    # maxLimit = 100
    from joblib import Parallel, delayed
    output_result_file = f'./dataset/WikiTableQuestions/results/CodexAnswerCOTExecutor_decisionAction_{template}_{program}_NNDemo={NNDemo}_results_pristine-unseen-tables.json'
    logs = Parallel(n_jobs=n_threads, require='sharedmem')(delayed(parallel_codex_func_formatv1)(i) for i in tqdm(range(min(maxLimit, dataset.shape[0]))))
    json.dump(logs, open(output_result_file, 'w'), indent=4)
    # evaluate: 
    os.system(f'cd ./dataset/WikiTableQuestions/ && python2 evaluator.py ./results/{output_result_file.split("/")[-1]} && cd ..')

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4344/4344 [2:39:38<00:00,  2.21s/it]
Reading dataset from ./tagged/data/training.tagged
Reading dataset from ./tagged/data/pristine-seen-tables.tagged
Reading dataset from ./tagged/data/pristine-unseen-tables.tagged
Read 22033 examples
Reading predictions from ./results/CodexAnswerCOTExecutor_decisionAction_original-sql-py_sql-py_NNDemo=False_results_pristine-unseen-tables.json
Examples: 4344
Correct: 2833
Accuracy: 0.6522
GPT error: 0.0
Uncaught error: 0.0497
{u'Gpt connection timeo': [u'nu-63', u'nu-116', u'nu-123', u'nu-153', u'nu-213', u'nu-222', u'nu-223', u'nu-246', u'nu-301', u'nu-310', u'nu-338', u'nu-344', u'nu-362', u'nu-376', u'nu-392', u'nu-395', u'nu-406', u'nu-422', u'nu-

In [47]:
NNDemo = False
max_demo = 5
template = 'original-sql-py'
program = 'sql-py'
def func(i):
    log = {}
    codex_prompter = CodexAnswerCOTExecutor_decisionAction(
                                      f'prompt_template/{template}-DA.json',
                                      dataset.iloc[i]['id'], 
                                      dataset.iloc[i]['utterance'], 
                                      dataset.iloc[i]['context'], 
                                      dataset.iloc[i]['targetValue'],  
                                      base_path='./dataset/WikiTableQuestions/',
                                      demo_file=f'few-shot-demo/WikiTQ-{program}.json',
                                     )
    codex_prompter.max_demo = max_demo
    codex_prompter._gen_gpt_prompt()
    # codex_prompter._gen_gpt_prompt(NNDemo)
    codex_prompter._get_gpt_prediction()
    log = codex_prompter._log_dict()
    return log, codex_prompter
log, codex_prompter = func(0)

In [49]:
# print(log['action_prompts'][1])
print(log['prompt'])
# print(log['decision_prompts'][1])
# print(codex_prompter.demo_prompts_by_decision['Python'])
# log

The database table DF is shown as follows:
[HEAD]: name|c_1989|c_1990|c_1991|c_1992|c_1993|c_1994|c_1995|c_1996|c_1997|c_1998|c_1999|c_2000|c_2001|c_2002|c_2003|c_2004|c_2005|c_2006|c_2007|c_2008|c_2009|c_2010|career_sr|career_win_loss
---
[ROW] 1: Australian Open|A|A|1R|A|2R|3R|2R|1R|A|3R|4R|1R|2R|1R|3R|2R|1R|QF|3R|2R|3R|1R|0 / 18|22–18
[ROW] 2: French Open|1R|2R|4R|1R|1R|3R|1R|A|1R|3R|1R|2R|4R|2R|2R|3R|1R|1R|1R|2R|1R|A|0 / 20|17–20
[ROW] 3: Wimbledon|A|1R|A|A|A|A|1R|A|1R|A|2R|2R|3R|2R|2R|2R|2R|2R|2R|1R|2R|A|0 / 14|11–14
...
[ROW] 17: Annual Win-Loss|nan|2–4|7–5|3–5|6–4|2–1|5–4|2–1|12–6|10–9|10–7|12–9|13–9|9–9|2–7|8–5|7–7|3–8|4–3|2–3|1–2|0–0|nan|120–108
[ROW] 18: Year End Ranking|235|62|43|43|55|46|102|118|29|41|34|31|22|35|62|52|58|52|37|52|68|–|nan|nan

Answer the following question based on the data above: "did he win more at the australian open or indian wells?". Execute SQL or Python code step-by-step and finally answer the question. Use SQL to process the query and use Python to